# সূচিপত্র
- [গাইডেন্স সম্পর্কে](../../../../code/01.Introduce)
- [সেটআপ](../../../../code/01.Introduce)
- [অবাধ জেনারেশন](../../../../code/01.Introduce)
- [ফি ৩ এর পক্ষে কথা বলা](../../../../code/01.Introduce)
- [রেগেক্স](../../../../code/01.Introduce)
- [নির্বাচন](../../../../code/01.Introduce)
- [চিন্তার শৃঙ্খল](../../../../code/01.Introduce)
- [JSON জেনারেশন](../../../../code/01.Introduce)
- [HTML জেনারেশন](../../../../code/01.Introduce)


# গাইডেন্স সম্পর্কে
গাইডেন্স একটি প্রমাণিত ওপেন-সোর্স পাইথন লাইব্রেরি যা যেকোনো ভাষার মডেলের (LM) আউটপুট নিয়ন্ত্রণ করতে সক্ষম। একটি API কলের মাধ্যমে, আপনি (পাইথনে) সুনির্দিষ্ট প্রোগ্রাম্যাটিক সীমাবদ্ধতা প্রকাশ করতে পারেন যা মডেলকে অনুসরণ করতে হবে এবং JSON, Python, HTML, SQL, বা যেকোনো কাঠামো যা আপনার ব্যবহারের ক্ষেত্রে প্রয়োজন তা অনুযায়ী গঠিত আউটপুট তৈরি করতে পারেন।

গাইডেন্স প্রচলিত প্রম্পটিং কৌশল থেকে ভিন্ন। এটি সীমাবদ্ধতা প্রয়োগ করে মডেলকে টোকেন বাই টোকেন ইনফারেন্স স্তরে পরিচালিত করে, যা উচ্চমানের আউটপুট তৈরি করে এবং অত্যন্ত গঠিত পরিস্থিতিতে ব্যবহার করার সময় খরচ এবং বিলম্ব ৩০–৫০% পর্যন্ত কমিয়ে দেয়।

গাইডেন্স সম্পর্কে আরও জানতে, [GitHub-এ পাবলিক রিপোজিটরি](https://github.com/guidance-ai/guidance) দেখুন অথবা Microsoft Build-এ [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) দেখুন।


# সেটআপ
1. `pip install guidance --pre` ব্যবহার করে Guidance ইনস্টল করুন
2. Azure-এ একটি Phi 3.5 mini endpoint ডিপ্লয় করুন এই লিঙ্কে গিয়ে: https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml এবং "Deploy" বোতামে ক্লিক করুন
3. আপনার endpoint-এর API key একটি `AZURE_PHI3_KEY` নামক environment variable-এ সংরক্ষণ করুন এবং URL একটি `AZURE_PHI3_URL` নামক environment variable-এ সংরক্ষণ করুন


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# অসীম জেনারেশন
`gen()` ফাংশন ব্যবহার করে কোনো সীমাবদ্ধতা ছাড়াই টেক্সট তৈরি করা যায়। এটি Guidance ছাড়া মডেল ব্যবহার করার মতোই।

## চ্যাট ফরম্যাটিং
অনেক চ্যাট মডেলের মতো, Phi-3 ব্যবহারকারী এবং সহকারীর মধ্যে বার্তাগুলিকে একটি নির্দিষ্ট ফরম্যাটে আশা করে। Guidance Phi-3 এর চ্যাট টেমপ্লেট সমর্থন করে এবং আপনার জন্য চ্যাট ফরম্যাটিং পরিচালনা করবে। চ্যাট টার্ন তৈরি করতে, কথোপকথনের প্রতিটি অংশকে `with user()` বা `with assistant()` ব্লকে রাখুন। একটি `with system()` ব্লক ব্যবহার করে সিস্টেম বার্তা সেট করা যেতে পারে।


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## টোকেন সঞ্চয়
খুবই গঠিত পরিস্থিতিতে, Guidance টোকেন এড়িয়ে যেতে পারে এবং শুধুমাত্র প্রয়োজনীয় টোকেন তৈরি করতে পারে, যা কর্মক্ষমতা উন্নত করে, দক্ষতা বৃদ্ধি করে এবং API খরচ সাশ্রয় করে। তৈরি করা টোকেনগুলি এই নোটবুকে হাইলাইট করা ব্যাকগ্রাউন্ড সহ দেখানো হয়েছে। বাধ্যতামূলক টোকেনগুলি হাইলাইট ছাড়া দেখানো হয় এবং ইনপুট টোকেনের মতোই খরচ হয়, যা আউটপুট টোকেনের খরচের এক তৃতীয়াংশ হিসাবে অনুমান করা হয়।

*নোট:* প্রথম উদাহরণটি যেখানে সীমাবদ্ধতা ছাড়া তৈরি করা হয়েছিল, কোনো টোকেন বাধ্য করতে সক্ষম হয়নি কারণ আমরা কোনো সীমাবদ্ধতা প্রদান করিনি।


# ফাই ৩ এর জন্য কথা বলা  
গাইডেন্স ব্যবহার করে আপনি সহজেই মডেলের প্রতিক্রিয়ায় টেক্সট যোগ করতে পারেন। এটি সহায়ক হতে পারে যদি আপনি মডেলের আউটপুটকে একটি নির্দিষ্ট দিকে পরিচালিত করতে চান।


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# রেগুলার এক্সপ্রেশনের মাধ্যমে সীমাবদ্ধ করা  
পূর্বের উদাহরণে, ফাই ৩ প্রশ্নের উত্তর `Canberra` দেওয়ার পর অনুসরণকারী ব্যাখ্যা প্রদান করেছিল। মডেলের আউটপুটকে ঠিক একটি শব্দে সীমাবদ্ধ করতে, একটি রেগুলার এক্সপ্রেশন ব্যবহার করা যেতে পারে।


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

রেজেক্সের মাধ্যমে শুধুমাত্র শব্দ `Canberra` তৈরি হয়।


# একাধিক বিকল্প থেকে নির্বাচন করা
যখন কিছু সম্ভাব্য বিকল্প জানা থাকে, আপনি `select()` ফাংশন ব্যবহার করতে পারেন যাতে মডেল একটি বিকল্প তালিকা থেকে নির্বাচন করতে পারে।


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` দিয়ে শুধুমাত্র টোকেন `Can` তৈরি করা হয়েছিল। কারণ `Canberra` একমাত্র বিকল্প যা সম্ভবত উত্তর সম্পূর্ণ করতে পারে, বাকি টোকেনগুলো বাধ্যতামূলকভাবে যোগ করা হয়েছিল।


# চিন্তার শৃঙ্খল
চিন্তার শৃঙ্খল একটি কৌশল যা মডেলের আউটপুটের গুণমান উন্নত করতে সাহায্য করে, এটি সমস্যাকে ধাপে ধাপে প্রক্রিয়াকরণে উৎসাহিত করে। সাধারণত, চূড়ান্ত উত্তরে পৌঁছানোর জন্য একাধিক প্রম্পট টার্ন প্রয়োজন হয়। প্রথমে, মডেলকে ধাপে ধাপে চিন্তা করতে নির্দেশ দিন। তারপর, মডেলকে আবার প্রম্পট দিন চূড়ান্ত উত্তর দেওয়ার জন্য। স্ট্যান্ডার্ড চ্যাট ইনফারেন্স API-গুলির মাধ্যমে, এটি ২টি API কল লাগে, এবং মডেলের তৈরি করা "চিন্তার শৃঙ্খল" দুইবার চার্জ হয় – একবার আউটপুট টোকেন হিসেবে যখন মডেল এটি তৈরি করে, এবং তারপর আবার ইনপুট টোকেন হিসেবে দ্বিতীয় কলের জন্য। Guidance-এর সাহায্যে, পুরো বহু-ধাপ প্রক্রিয়াটি একটি API কলের অংশ হিসেবে প্রক্রিয়াকৃত এবং চার্জ হয়, যা খরচ এবং বিলম্ব কমায়।


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON জেনারেশন
Guidance ব্যবহার করে JSON স্কিমা বা pydantic মডেলের সাথে সামঞ্জস্যপূর্ণ JSON তৈরি নিশ্চিত করা যায়, যেমন এখানে প্রদর্শিত ব্যবহারকারীর প্রোফাইল স্কিমা।


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML জেনারেশন

Guidance ব্যবহার করে কোড তৈরি করা এবং প্রোগ্রামিং ভাষার সিনট্যাক্সিকাল প্রয়োজনীয়তাগুলি অনুসরণ করা সম্ভব। এই অংশে, আমরা একটি ছোট Guidance প্রোগ্রাম তৈরি করব যা খুব সাধারণ HTML ওয়েবপেজ লেখার জন্য ব্যবহৃত হবে।

আমরা ওয়েবপেজটিকে ছোট ছোট অংশে ভাগ করব, প্রতিটি অংশের জন্য আলাদা Guidance ফাংশন থাকবে। এরপর এই ফাংশনগুলোকে একত্রিত করে আমাদের চূড়ান্ত ফাংশন তৈরি করব, যা একটি HTML ওয়েবপেজ তৈরি করবে। 
এরপর আমরা এই ফাংশনটি Azure AI-তে Guidance-সক্ষম মডেলের বিরুদ্ধে চালাব।

*নোট:* এটি একটি পূর্ণাঙ্গ HTML জেনারেটর হবে না; এর উদ্দেশ্য হলো দেখানো যে কীভাবে আপনি আপনার নির্দিষ্ট প্রয়োজনের জন্য কাঠামোবদ্ধ আউটপুট তৈরি করতে পারেন।

আমরা Guidance থেকে যা প্রয়োজন তা আমদানি করে শুরু করব:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML ওয়েবপেজগুলি অত্যন্ত গঠনমূলক, এবং আমরা Guidance ব্যবহার করে পৃষ্ঠার সেই অংশগুলিকে 'জোর' করব।  
যখন আমরা মডেল থেকে স্পষ্টভাবে টেক্সট চাই, তখন নিশ্চিত করতে হবে যে এতে এমন কিছু অন্তর্ভুক্ত না হয় যা ট্যাগ হতে পারে - অর্থাৎ, আমাদের '<' এবং '>' অক্ষরগুলি বাদ দিতে হবে।


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

আমরা তারপর এই ফাংশনটি ব্যবহার করে একটি ইচ্ছামত HTML ট্যাগের মধ্যে টেক্সট তৈরি করতে পারি:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

এখন, চলুন আমরা পৃষ্ঠার শিরোনাম তৈরি করি।  
এর অংশ হিসেবে, আমাদের একটি পৃষ্ঠার শিরোনাম তৈরি করতে হবে:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML পৃষ্ঠার মূল অংশ শিরোনাম এবং অনুচ্ছেদ দিয়ে পূর্ণ হবে।  
আমরা প্রতিটির জন্য একটি ফাংশন সংজ্ঞায়িত করতে পারি:  


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

এখন, HTML এর মূল অংশ সংজ্ঞায়িত করার জন্য ফাংশনটি।

এটি `select()` ব্যবহার করে `recurse=True` সহ একাধিক শিরোনাম এবং অনুচ্ছেদ তৈরি করে:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

এরপর আমরা সেই ফাংশনের দিকে আসি যা সম্পূর্ণ HTML পৃষ্ঠা তৈরি করে।  
আমরা HTML শুরুর ট্যাগ যোগ করি, তারপর হেডার তৈরি করি, তারপর বডি, এবং শেষে HTML শেষ ট্যাগ যোগ করি:  


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

আমরা একটি ব্যবহারকারী-বান্ধব র‍্যাপার প্রদান করি, যা আমাদেরকে নিম্নলিখিত কাজগুলো করতে সক্ষম করবে:
- জেনারেশনের তাপমাত্রা নির্ধারণ করা
- মডেল অবজেক্ট থেকে তৈরি হওয়া পৃষ্ঠাটি সংগ্রহ করা


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

আমরা তারপর আউটপুটটি একটি ফাইলে লিখতে পারি:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

এবং [ফলাফল দেখুন](../../../../code/01.Introduce/sample_page.html)।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
